<a href="https://colab.research.google.com/github/KavyaD02/Continuous_Speech_Stress_Detection/blob/main/Stress_Level_Diagnosis_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
import librosa.display
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects 
from scipy.io.wavfile import read, write

C:\Users\Kavya Duvvuri\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


def preprocess_audio(audio_file_name):

    audio_file, sr = librosa.load(audio_file_name)
    
    audio_file = librosa.effects.preemphasis(audio_file)
    audio_file = librosa.effects.trim(audio_file, top_db=20)[0]
    audio_file = librosa.util.normalize(audio_file)
    
    return sr, audio_file

In [ ]:
DATA_PATH = "./Data/"

def get_file_names(path = DATA_PATH):
    file_names = os.listdir(path)
    return file_names

def process_audio(audio_file_name):
    rate, audio = read(f"./Data/{audio_file_name}")

    aud = AudioSegment(audio.tobytes(), frame_rate = rate,
                         sample_width = audio.dtype.itemsize, channels = 1)

    audio_chunks = split_on_silence(
        aud,
        min_silence_len = 1000,
        silence_thresh = -50,
        keep_silence = 500,)
    
    # audio chunks are combined here
    audio_processed = sum(audio_chunks)
    audio_processed = np.array(audio_processed.get_array_of_samples())
    audio_file = audio_file_name.split(".")[0]
    write(f"./Processed Data/{audio_file}_PROCESSED.wav", rate, audio_processed)

file_names = get_file_names()
for file_name in file_names:
    process_audio(file_name)
print("Done")

In [ ]:
def get_label_dict(df, file_list, test=False):
    label_dict = {}
    
    if test:
        df_list = df['PHQ_Score']
    else:
        df_list = df['PHQ8_Score']

    for file in file_list:
        patient_num = int(file.split("/")[-1].split("_")[0])

        patient_list = list(df['Participant_ID'])

        idx = patient_list.index(patient_num)

        phq8_score = int(df_list[idx])
        
        if phq8_score in range(0, 6):
            score = 0
        elif phq8_score in range(6, 15):
            score = 1
        elif phq8_score in range(15, 22):
            score = 2
        elif phq8_score in range(22, 28):
            score = 3

        label_dict[f"{file}"] = score
    
    return label_dict

In [ ]:
def get_set(df):
    df_files = [f"./Processed Data/{x}_AUDIO_PROCESSED.wav" for x in df['Participant_ID']]

    return list(set(df_files))

In [ ]:
train_df, test_df, val_df = pd.read_csv(f"./train.csv"), pd.read_csv(f"./test.csv"), pd.read_csv(f"./val.csv")

train, test, val = get_set(train_df), get_set(test_df), get_set(val_df)

In [ ]:
train_dict, test_dict, val_dict = get_label_dict(train_df, train), get_label_dict(test_df, test, True), get_label_dict(val_df, val)
data_dict = {**train_dict, **test_dict, **val_dict}

In [ ]:
PROCESSED_DATA_PATH = "./Processed Data/"

In [ ]:
data_list = list(data_dict.keys())
label_list = list(data_dict.values())

In [ ]:
new_data_list = []
new_label_list = []
for data in data_list:
    audios = os.listdir(PROCESSED_DATA_PATH)
    data_2 = data.split('/')[2]
    if data_2 in audios:
        new_data_list.append(data)
        new_label_list.append(label_list[data_list.index(data)])

In [ ]:
csv_path = "./Combined Features.csv"
combined_df = pd.read_csv(csv_path)
combined = list(combined_df["Combined"])
combined_features = []
for i in combined:
    new_i = i.replace('\n', ' ')
    new_i = new_i.replace('[', '')
    new_i = new_i.replace(']', '')
    array_data = np.fromstring(new_i, dtype=np.float64, sep=' ')
    combined_features.append(array_data)
print(len(combined_features[0]))

64


In [ ]:
X = np.array(combined_features).reshape(186, 64, 1)

In [ ]:
Y = np.array(new_label_list)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.20)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(64,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y, epochs = 200, validation_data = (test_X, test_Y))

Epoch 1/200
5/5 [==============================] - 1s 53ms/step - loss: 0.7448 - accuracy: 0.6014 - val_loss: 1.2087 - val_accuracy: 0.4737
Epoch 2/200
5/5 [==============================] - 0s 11ms/step - loss: 0.6966 - accuracy: 0.6351 - val_loss: 1.2237 - val_accuracy: 0.4737
Epoch 3/200
5/5 [==============================] - 0s 11ms/step - loss: 0.7514 - accuracy: 0.6824 - val_loss: 1.3022 - val_accuracy: 0.4737
Epoch 4/200
5/5 [==============================] - 0s 11ms/step - loss: 0.7380 - accuracy: 0.6892 - val_loss: 1.2830 - val_accuracy: 0.4211
Epoch 5/200
5/5 [==============================] - 0s 14ms/step - loss: 0.6495 - accuracy: 0.6622 - val_loss: 1.2224 - val_accuracy: 0.4474
Epoch 6/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6658 - accuracy: 0.6824 - val_loss: 1.2492 - val_accuracy: 0.4474
Epoch 7/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6910 - accuracy: 0.6554 - val_loss: 1.3651 - val_accuracy: 0.3684
Epoch 8/200
5/5 [===

5/5 [==============================] - 0s 12ms/step - loss: 0.6049 - accuracy: 0.7500 - val_loss: 1.5924 - val_accuracy: 0.4737
Epoch 60/200
5/5 [==============================] - 0s 12ms/step - loss: 0.5252 - accuracy: 0.7500 - val_loss: 1.3386 - val_accuracy: 0.4211
Epoch 61/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6002 - accuracy: 0.6824 - val_loss: 1.3157 - val_accuracy: 0.5000
Epoch 62/200
5/5 [==============================] - 0s 12ms/step - loss: 0.5369 - accuracy: 0.7905 - val_loss: 1.5637 - val_accuracy: 0.4737
Epoch 63/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6264 - accuracy: 0.7095 - val_loss: 1.4719 - val_accuracy: 0.4211
Epoch 64/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6712 - accuracy: 0.7162 - val_loss: 1.3199 - val_accuracy: 0.5000
Epoch 65/200
5/5 [==============================] - 0s 12ms/step - loss: 0.6031 - accuracy: 0.7297 - val_loss: 1.3380 - val_accuracy: 0.4474
Epoch 66/200
5/5 [========

5/5 [==============================] - 0s 12ms/step - loss: 0.3819 - accuracy: 0.8514 - val_loss: 1.6817 - val_accuracy: 0.4211
Epoch 118/200
5/5 [==============================] - 0s 12ms/step - loss: 0.3951 - accuracy: 0.8514 - val_loss: 1.6036 - val_accuracy: 0.3684
Epoch 119/200
5/5 [==============================] - 0s 12ms/step - loss: 0.3964 - accuracy: 0.8108 - val_loss: 1.7772 - val_accuracy: 0.3158
Epoch 120/200
5/5 [==============================] - 0s 12ms/step - loss: 0.3093 - accuracy: 0.8784 - val_loss: 1.8007 - val_accuracy: 0.3947
Epoch 121/200
5/5 [==============================] - 0s 11ms/step - loss: 0.3945 - accuracy: 0.8176 - val_loss: 1.6589 - val_accuracy: 0.4737
Epoch 122/200
5/5 [==============================] - 0s 12ms/step - loss: 0.3380 - accuracy: 0.8514 - val_loss: 1.6586 - val_accuracy: 0.4474
Epoch 123/200
5/5 [==============================] - 0s 11ms/step - loss: 0.4210 - accuracy: 0.8311 - val_loss: 1.7772 - val_accuracy: 0.4737
Epoch 124/200
5/5 [=

Epoch 175/200
5/5 [==============================] - 0s 12ms/step - loss: 0.3144 - accuracy: 0.8716 - val_loss: 2.0703 - val_accuracy: 0.3421
Epoch 176/200
5/5 [==============================] - 0s 12ms/step - loss: 0.2597 - accuracy: 0.8986 - val_loss: 2.0348 - val_accuracy: 0.3684
Epoch 177/200
5/5 [==============================] - 0s 11ms/step - loss: 0.2992 - accuracy: 0.8446 - val_loss: 1.8629 - val_accuracy: 0.4474
Epoch 178/200
5/5 [==============================] - 0s 11ms/step - loss: 0.2577 - accuracy: 0.8986 - val_loss: 1.8955 - val_accuracy: 0.4211
Epoch 179/200
5/5 [==============================] - 0s 11ms/step - loss: 0.2705 - accuracy: 0.9189 - val_loss: 2.0520 - val_accuracy: 0.4474
Epoch 180/200
5/5 [==============================] - 0s 11ms/step - loss: 0.2669 - accuracy: 0.8919 - val_loss: 2.1130 - val_accuracy: 0.4474
Epoch 181/200
5/5 [==============================] - 0s 11ms/step - loss: 0.2547 - accuracy: 0.8851 - val_loss: 2.1005 - val_accuracy: 0.3947
Epoch 